In [1]:
%%writefile main.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

// Helper function for checking CUDA API errors
#define CHECK_CUDA(call) do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        fprintf(stderr, "CUDA Error at %s:%d - %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE); \
    } \
} while (0)

// Helper function for checking cuBLAS API errors
#define CHECK_CUBLAS(call) do { \
    cublasStatus_t status = call; \
    if (status != CUBLAS_STATUS_SUCCESS) { \
        fprintf(stderr, "cuBLAS Error at %s:%d - Status %d\n", __FILE__, __LINE__, status); \
        /* You might want a function to convert cublasStatus_t to string here */ \
        exit(EXIT_FAILURE); \
    } \
} while (0)


int main() {
    // --- Problem Definition ---
    // We want to compute y = A*x where A is 3x2, x is 2x1, y is 3x1.
    // A is stored as a submatrix of a larger 5x2 matrix (column-major).
    // x uses non-contiguous elements from a larger vector (size 3, incx=2).

    const int M = 3; // Rows of matrix A (and vector y)
    const int N = 2; // Columns of matrix A (and vector x)

    // --- Storage Definition ---
    const int lda = 5;        // Leading dimension of A (rows of the parent matrix)
    const int incx = 2;       // Increment for vector x
    const int incy = 1;       // Increment for vector y (contiguous)

    const int A_parent_rows = lda;
    const int A_parent_cols = N;
    const size_t A_alloc_size = (size_t)A_parent_rows * A_parent_cols; // Total elements in parent A storage

    // We need space for 1 + (N-1)*incx elements for x storage conceptually.
    // To keep it simple based on user request (vector length 3, incx 2 uses indices 0 and 2):
    const int x_alloc_count = 3;
    const size_t X_alloc_size = x_alloc_count; // Total elements in x storage

    const size_t Y_alloc_size = M; // Total elements in y storage (contiguous)


    // --- Host Data Initialization ---
    float *h_A, *h_x, *h_y;
    h_A = (float*)malloc(A_alloc_size * sizeof(float));
    h_x = (float*)malloc(X_alloc_size * sizeof(float));
    h_y = (float*)malloc(Y_alloc_size * sizeof(float));

    if (!h_A || !h_x || !h_y) {
        fprintf(stderr, "Failed to allocate host memory\n");
        return EXIT_FAILURE;
    }

    printf("Initializing host data...\n");
    // Initialize parent matrix A (Column Major!)
    // Conceptual 3x2 A is the top-left part:
    // [ 1.0  4.0 ]
    // [ 2.0  5.0 ]
    // [ 3.0  6.0 ]
    // Parent 5x2 matrix storage (Column Major):
    // Col 0: [ 1.0, 2.0, 3.0, 7.0, 8.0 ]  <- lda = 5
    // Col 1: [ 4.0, 5.0, 6.0, 9.0, 10.0] <- lda = 5
    float A_vals[A_alloc_size] = {
        1.0f, 2.0f, 3.0f, 7.0f, 8.0f,  // Column 0
        4.0f, 5.0f, 6.0f, 9.0f, 10.0f  // Column 1
    };
    for(size_t i = 0; i < A_alloc_size; ++i) h_A[i] = A_vals[i];

    // Initialize vector x storage
    // Conceptual x is [ 11.0, 12.0 ]
    // Stored in h_x with size 3 and incx = 2: uses h_x[0] and h_x[0 + 1*incx] = h_x[2]
    // h_x = [ 11.0, 99.0, 12.0 ] (99.0 is padding/unused)
    h_x[0] = 11.0f;
    h_x[1] = 99.0f; // This element will be skipped due to incx=2
    h_x[2] = 12.0f;

    // Initialize host y (optional, as beta=0)
    for (int i = 0; i < M; ++i) {
        h_y[i] = 0.0f;
    }

    // --- CUDA Initialization and Memory Allocation ---
    cublasHandle_t handle;
    float *d_A = NULL, *d_x = NULL, *d_y = NULL;

    printf("Creating cuBLAS handle...\n");
    CHECK_CUBLAS(cublasCreate(&handle));

    printf("Allocating device memory...\n");
    CHECK_CUDA(cudaMalloc((void**)&d_A, A_alloc_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc((void**)&d_x, X_alloc_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc((void**)&d_y, Y_alloc_size * sizeof(float)));

    // --- Copy Data Host to Device ---
    printf("Copying data from host to device...\n");
    CHECK_CUDA(cudaMemcpy(d_A, h_A, A_alloc_size * sizeof(float), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_x, h_x, X_alloc_size * sizeof(float), cudaMemcpyHostToDevice));
    // No need to copy h_y if beta is 0, but do it if beta != 0 or for consistency
    CHECK_CUDA(cudaMemcpy(d_y, h_y, Y_alloc_size * sizeof(float), cudaMemcpyHostToDevice));

    // --- Execute cuBLAS Sgemv ---
    printf("Executing cublasSgemv...\n");
    const float alpha = 1.0f;
    const float beta = 0.0f;

    // cublasStatus_t cublasSgemv(cublasHandle_t handle,
    //                            cublasOperation_t trans,
    //                            int m, int n,
    //                            const float *alpha,
    //                            const float *A, int lda,
    //                            const float *x, int incx,
    //                            const float *beta,
    //                            float *y, int incy);
    CHECK_CUBLAS(cublasSgemv(handle,
                             CUBLAS_OP_N,    // No transpose of A
                             M,              // Rows of A (3)
                             N,              // Cols of A (2)
                             &alpha,         // Scalar alpha (1.0)
                             d_A,            // Pointer to device matrix A
                             lda,            // Leading dimension of A (5)
                             d_x,            // Pointer to device vector x
                             incx,           // Increment for x (2)
                             &beta,          // Scalar beta (0.0)
                             d_y,            // Pointer to device vector y
                             incy            // Increment for y (1)
                             ));

    // --- Copy Result Device to Host ---
    printf("Copying result from device to host...\n");
    CHECK_CUDA(cudaMemcpy(h_y, d_y, Y_alloc_size * sizeof(float), cudaMemcpyDeviceToHost));

    // --- Cleanup ---
    printf("Cleaning up...\n");
    CHECK_CUDA(cudaFree(d_A));
    CHECK_CUDA(cudaFree(d_x));
    CHECK_CUDA(cudaFree(d_y));
    CHECK_CUBLAS(cublasDestroy(handle));

    // --- Print Result ---
    printf("\n--- Input --- \n");
    printf("Matrix A (3x2 submatrix, column major, lda=%d):\n", lda);
    printf("[ %5.1f %5.1f ]\n", h_A[0], h_A[lda*1 + 0]); // A[0,0], A[0,1]
    printf("[ %5.1f %5.1f ]\n", h_A[1], h_A[lda*1 + 1]); // A[1,0], A[1,1]
    printf("[ %5.1f %5.1f ]\n", h_A[2], h_A[lda*1 + 2]); // A[2,0], A[2,1]
    // printf("Full A storage (Column Major):\n");
    // for(int j=0; j<N; ++j) {
    //     for(int i=0; i<lda; ++i) {
    //         printf("%5.1f ", h_A[j*lda + i]);
    //     }
    //     printf("\n");
    // }


    printf("\nVector x (Effective size %d, stored size %d, incx=%d):\n", N, x_alloc_count, incx);
    printf("[ %5.1f ] (from h_x[0])\n", h_x[0]);
    printf("[ %5.1f ] (from h_x[2])\n", h_x[2]);
    // printf("Full x storage: [ ");
    // for(int i=0; i<x_alloc_count; ++i) printf("%5.1f ", h_x[i]);
    // printf("]\n");


    printf("\n--- Output --- \n");
    printf("Result vector y (size %d, incy=%d):\n", M, incy);
    printf("[ ");
    for (int i = 0; i < M; ++i) {
        printf("%8.2f ", h_y[i]);
    }
    printf("]\n");

    // Expected result:
    // y[0] = 1.0*11.0 + 4.0*12.0 = 11 + 48 = 59
    // y[1] = 2.0*11.0 + 5.0*12.0 = 22 + 60 = 82
    // y[2] = 3.0*11.0 + 6.0*12.0 = 33 + 72 = 105
    printf("\nExpected result:\n[   59.00    82.00   105.00 ]\n");


    // Free host memory
    free(h_A);
    free(h_x);
    free(h_y);

    printf("\nDone.\n");
    return EXIT_SUCCESS;
}

Writing main.cu


In [4]:
!nvcc -arch=sm_70 main.cu -o main.ex -lcublas

In [5]:
!./main.ex

Initializing host data...
Creating cuBLAS handle...
Allocating device memory...
Copying data from host to device...
Executing cublasSgemv...
Copying result from device to host...
Cleaning up...

--- Input --- 
Matrix A (3x2 submatrix, column major, lda=5):
[   1.0   4.0 ]
[   2.0   5.0 ]
[   3.0   6.0 ]

Vector x (Effective size 2, stored size 3, incx=2):
[  11.0 ] (from h_x[0])
[  12.0 ] (from h_x[2])

--- Output --- 
Result vector y (size 3, incy=1):
[    59.00    82.00   105.00 ]

Expected result:
[   59.00    82.00   105.00 ]

Done.
